In [1]:
# for all execution
!mkdir -p local_logs

In [40]:
# import os

# os.environ['ACCESS_KEY'] = 'DKOBSROD'
# os.environ['SECRET_KEY'] = 'd4gdmpfeorhbt72o'


In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')


In [16]:
!sacctmgr show user $USER withassoc



      User   Def Acct     Admin    Cluster    Account  Partition     Share   Priority MaxJobs MaxNodes  MaxCPUs MaxSubmit     MaxWall  MaxCPUMins                  QOS   Def QOS 
---------- ---------- --------- ---------- ---------- ---------- --------- ---------- ------- -------- -------- --------- ----------- ----------- -------------------- --------- 
      eila   akundaje      None   sherlock   akundaje                  100                                                                               high_p,normal    normal 


In [5]:
# !echo $ACCESS_KEY
# !echo $SECRET_KEY

In [18]:
# check the jobs:

In [1]:
!squeue -u eila


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [20]:
# !squeue -u eila

In [135]:
# !squeue -u eila

In [136]:
# !scancel -u eila

In [137]:
# !squeue -u eila --name=step1-do


In [138]:
# !chmod +x execute_sbatch_arrays_on_sherlock.py 

# eila_in_download_bams.txt: The input file name.
# step1-download-bams.sh: The SLURM script name.
# 1: entries_per_job (number of entries per job). - how much is each job needs
# 1: max_concurrent_tasks (maximum number of concurrent tasks). - 500 for stanford


# example for input of 1300 lines:
# ./execute_sbatch_arrays_on_sherlock.sh eila_in_download_bams.txt step1-download-bams.sh 500 500
# Breakdown:
# eila_in_download_bams.txt: The input file with 1200 lines.
# step1-download-bams.sh: The SLURM script to run.
# 500: Number of lines per job array (tasks per array).
# 500: Maximum number of concurrent tasks per array.

In [9]:
# step 1
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')

# !echo $ACCESS_KEY
# !echo $SECRET_KEY

# rm slurm-*.out
# !scancel -u eila
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/eila_encids_with_bam_urls.txt step1-download-bams.sh 1000 1000

# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step1/eila_encids_with_bam_urls-debug.txt step1-download-bams.sh 2 2


In [10]:
# #step 2 - samtools sort and index
# !ls /scratch/users/eila/encode_pseudobulks_data/bam/ENCSR*/*unsorted.bam > bam_files_list.txt

# !./execute_sbatch_arrays_on_sherlock.sh bam_files_list.txt step2-samtool-sort-index.sh 100 100

In [107]:
#step 3 - peaks - independant of BAMS:

# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step3/eila_peaks_matched_to_experiments.txt step3-download-peaks.sh 1000 1000

# debug
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step3/eila_peaks_matched_to_experiments-debug.txt step3-download-peaks.sh 2 2




In [18]:
# !pip install pandas

In [19]:
# # input file for step4:

# import pandas as pd

# # Read the species file
# species_df = pd.read_csv('./steps_inputs/encids_matched_to_species.txt', sep='\t', header=None, names=['ID', 'Species'])

# # Read the data file
# data_df = pd.read_csv('./steps_inputs/step4/peaks_matched_to_experiments.txt', sep=' ', header=None, names=['ID', 'ENCFF', 'Type', 'URL'])

# # Merge the data files based on the ID
# merged_df = pd.merge(data_df, species_df, left_on='ID', right_on='ID', how='left')

# # Save the merged data to a new file
# merged_df.to_csv('./steps_inputs/step4/peaks_matched_to_experiments_species.txt', sep=' ', index=False, header=False)



In [90]:
# #step 4 - non-peaks / background / negative - human:

# debug
# !rm ./local_logs/slurm_step4_negative_*
# !rm -r $SCRATCH/encode_pseudobulks_negative
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step4/debug_peaks_matched_to_experiments.txt step4-compute-negatives-non-peak-background.sh 2 2

# ## all samples
# ## !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step4/peaks_matched_to_experiments_species.txt step4-compute-negatives-non-peak-background.sh 1000 1000


In [15]:
!rm ./local_logs/slurm_step5*


In [11]:
# prep for step 5
# !chmod +x step5-1-create_input.sh
# !./step5-1-create_input.sh
# !rm ./local_logs/slurm_step5*
!./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/debug-extracted_paths.txt step5-2-train-chrombonet-model.sh 2 2


# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/extracted_paths.csv step5-2-train-chrombonet-model.sh 1000 1000


Total entries in input file: 3
Total number of job arrays to be submitted: 2
Submitting job array from 1 to 2 with a maximum of 2 concurrent tasks.
sbatch: defined options
sbatch: -------------------- --------------------
sbatch: array               : 1-2%2
sbatch: gpus                : 1
sbatch: job-name            : step5-train-chrombpnet
sbatch: mem                 : 64G
sbatch: ntasks              : 1
sbatch: partition           : akundaje,owners
sbatch: time                : 2-00:00:00
sbatch: verbose             : 1
sbatch: -------------------- --------------------
sbatch: end of defined options
Submitted batch job 52710590
Submitting job array from 3 to 3 with a maximum of 2 concurrent tasks.
sbatch: defined options
sbatch: -------------------- --------------------
sbatch: array               : 3-3%2
sbatch: gpus                : 1
sbatch: job-name            : step5-train-chrombpnet
sbatch: mem                 : 64G
sbatch: ntasks              : 1
sbatch: partition           : 

In [85]:
# !sstat -j 52658719 --format=JobID,MaxRSS,MaxVMSize


In [117]:
# !scontrol show job 52659563


In [4]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
        52707140_1  akundaje step5-tr     eila  R      14:45      1 sh03-16n09
        52707140_2 owners,ak step5-tr     eila PD       0:00      1 (Priority)


In [5]:
!squeue -u eila --state=FAILED,PREEMPTED,TIMEOUT,PREEMPTED,NODE_FAIL,CANCELLED

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
        52707144_3    owners step5-tr     eila  F       0:05      1 (NonZeroExitCode)


In [7]:
!pwd

/oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas


In [109]:
# !scancel -u eila

In [14]:
# !scancel 52710582

In [15]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
        52707140_1  akundaje step5-tr     eila  R      47:19      1 sh03-16n09
        52709857_1    owners step5-tr     eila  R      11:57      1 sh03-11n11
        52709857_2    owners step5-tr     eila  R      11:57      1 sh03-15n07
        52707140_2    owners step5-tr     eila  R      21:15      1 sh03-11n10
    52710591_[3%2] owners,ak step5-tr     eila PD       0:00      1 (Resources)
  52710590_[1-2%2] owners,ak step5-tr     eila PD       0:00      1 (Resources)
  52710581_[1-2%2] owners,ak step5-tr     eila PD       0:00      1 (Resources)


In [99]:
# #step 5 - train chrombpnet - human

# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step5/human/eila_peaks_matched_to_experiments-debug.txt step4-compute-negatives-non-peak-background.sh 2 2




In [18]:
# !ls /scratch/users/eila/encode_pseudobulks/chrombpnet_model

In [19]:
# !ls /scratch/users/eila/encode_pseudobulks/chrombpnet_model/ENCSR396HHA


In [20]:
# !ls /scratch/users/eila/encode_pseudobulks/chrombpnet_model/ENCSR396HHA/negatives/fold_0


In [23]:
# !scontrol show job 52237837
!squeue --user=$USER --state=PENDING | wc -l

3


In [24]:
!squeue --user=$USER --state=COMPLETED | wc -l

1


In [26]:
# !squeue --user=$USER --state=RUNNING

In [27]:
!squeue --user=$USER --state=PENDING


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
52575521_[418-1000 owners,ak step4_ca     eila PD       0:00      1 (Priority)
52575522_[1073-135 owners,ak step4_ca     eila PD       0:00      1 (Priority)


In [28]:
!echo $USER

eila


In [108]:
# !scontrol show job 52275013


In [54]:
# !squeue -u eila


In [55]:
# !squeue --user=$USER --state=PENDING


In [56]:
# !squeue --user=$USER --state=RUNNING


In [87]:
# !scontrol show job | grep $USER
!squeue -u eila --state=FAILED,PREEMPTED,TIMEOUT,PREEMPTED,NODE_FAIL,CANCELLED

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
        52526441_1  akundaje step4_ca     eila  F       0:30      1 (NonZeroExitCode)


In [156]:
# !squeue -u eila -o "%.18i %.9P %.12j %.8u %.2t %.10M %.10l %.10Q %.19V %.19S"


In [75]:
!pwd

/oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas


In [ ]:
# !squeue -u eila

In [22]:
# Remove logs
# !find ./local_logs/ -name '*slurm*' -type f -mmin +600 -exec rm {} \;


In [52]:
# !ls /scratch/users/eila/encode_pseudobulks/ENCSR000NVR/bam/ENCSR000NVR_unsorted.bam

In [59]:
# !ls /scratch/users/eila/encode_pseudobulks/ENCSR000NVR/peaks/ENCFF585MYS/ENCSR000NVR_ENCFF585MYS.bed.gz


/scratch/users/eila/encode_pseudobulks/ENCSR000NVR/peaks/ENCFF585MYS/ENCSR000NVR_ENCFF585MYS.bed.gz


In [3]:
!ls /scratch/users/eila/encode_pseudobulks/ENCSR*/bam/ENCSR*_unsorted.bam | wc -l

1355


In [4]:
!ls /scratch/users/eila/encode_pseudobulks/ENCSR*/bam/ENCSR*_sorted.bam | wc -l

1355


In [5]:
!ls /scratch/users/eila/encode_pseudobulks/ENCSR*/peaks/*/*.bed.gz | wc -l


1355


In [ ]:
!ls /scratch/users/eila/encode_pseudobulks/ENCSR*/bam/ENCSR*_unsorted.bam | wc -l

In [ ]:
!ls /scratch/users/eila/encode_pseudobulks/ENCSR*/bam/ENCSR*_sorted.bam | wc -l

In [113]:
# !rm ./local_logs/slurm_step1_download_bams_err.combined.err

In [13]:
# !rm ./local_logs/slurm_step4*

In [2]:
!ls

bam_files_list.txt
conda_chrombpnet_installation_cmds.txt
execute_sbatch_arrays_on_sherlock.sh
local_logs
shared_vars.sh
SLURM_CMDS.ipynb
step1-download-bams.sh
step2-samtool-sort-index.sh
step3-download-peaks.sh
step4-compute-negatives-non-peak-background.sh
step5-train-chrombonet-model.sh
steps_inputs


In [5]:
!pwd

/oak/stanford/groups/akundaje/eila/git_tf_chromatin_atlas


In [6]:
!ls -l ./local_logs/slurm_step4*

-rw-rw----+ 1 eila oak_akundaje 10224 Sep  4 09:36 ./local_logs/slurm_step4_negative_err.combined.err
-rw-rw----+ 1 eila oak_akundaje  9916 Sep  4 09:36 ./local_logs/slurm_step4_negative_out.combined.out


In [ ]:
# bias - given for the train
# train - on one file. provides correlation metrics. we need to look at the motifs that modisco populates
#         model. modisco on subset of the peaks- subset is random. evaludation folder - modisco and interpret on 30K areas
# QC
# interpret on subset of the peaks.


In [ ]:
!echo $HOME

In [15]:
# %%bash

# # Define the base directories
# SRC_BASE="/scratch/users/eila/encode_pseudobulks_data"
# DEST_BASE="/scratch/users/eila/encode_pseudobulks_data/bam"

# # Loop through all directories matching ENCSR* and move files
# for DIR in ${SRC_BASE}/ENCSR*/bam; do
#     # Extract the directory name after /ENCSR
#     DIR_NAME=$(basename $(dirname "$DIR"))
    
#     # Define source and destination directories
#     SRC_DIR="${SRC_BASE}/${DIR_NAME}/bam"
#     DEST_DIR="${DEST_BASE}/${DIR_NAME}"
    
#     # Create destination directory if it doesn't exist
#     mkdir -p "$DEST_DIR"
    
#     # Move files from source to destination
#     mv "$SRC_DIR"/* "$DEST_DIR"/
    
#     echo "Moved files from $SRC_DIR to $DEST_DIR"
# done
